In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
1,application_1607949680860_0002,pyspark,idle,Link,Link


SparkSession available as 'spark'.

In [2]:
import hashlib
from datetime import datetime
from pyspark.sql import functions as func
from pyspark.sql.types import FloatType
import hsfs

In [3]:
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

## Placeholder

In [4]:
transactions_fg = fs.get_feature_group("transactions_fg", 1)

In [5]:
transactions_df = transactions_fg.read()
transactions_df.show()

+--------------+------+------+-------+------+--------+-------+--------+
|tran_timestamp|source|is_sar|tran_id|target|alert_id|tx_type|base_amt|
+--------------+------+------+-------+------+--------+-------+--------+
|   1.4832288E9|  1767|     0|      1|  3259|      -1|      4| 9405.71|
|   1.4832288E9|  7363|     0|      2|  5141|      -1|      4| 6884.54|
|   1.4832288E9|  7585|     0|      3|  9532|      -1|      4|  7968.4|
|   1.4832288E9|  1750|     0|      4|  8792|      -1|      4| 9042.67|
|   1.4832288E9|  9060|     0|      5|  4670|      -1|      4| 4692.79|
|   1.4832288E9|  8752|     0|      6|  3861|      -1|      4| 4089.65|
|   1.4832288E9|  9645|     0|      7|  3805|      -1|      4| 3055.04|
|   1.4832288E9|  7150|     0|      8|  9280|      -1|      4| 6473.45|
|   1.4832288E9|  1685|     0|      9|  6369|      -1|      4| 2583.42|
|   1.4832288E9|  7846|     0|     10|  8255|      -1|      4| 6753.04|
|   1.4832288E9|   878|     0|     11|  5957|      -1|      4| 1

## Placeholder

In [6]:
edges_td_meta = fs.create_training_dataset(name="edges_td",
                                       version=1,
                                       data_format="csv",
                                       label = ["is_sar"],   
                                       description="edges training dataset")
edges_td_meta.save(transactions_df)

### Placeholder

In [7]:
account_fg = fs.get_feature_group("account_features", 1)

In [8]:
accounts_df = account_fg.read()
accounts_df.show()

+--------------+---------------+---------+-------+
|tx_behavior_id|initial_deposit|prior_sar|acct_id|
+--------------+---------------+---------+-------+
|             1|       84442.19|        0|      0|
|             1|       75795.44|        0|      1|
|             1|       42057.16|        0|      2|
|             1|       25891.68|        0|      3|
|             1|       51127.47|        0|      4|
|             1|       40493.41|        0|      5|
|             1|       78379.86|        0|      6|
|             1|       30331.27|        0|      7|
|             1|        47659.7|        0|      8|
|             1|        58338.2|        0|      9|
|             1|       90811.29|        0|     10|
|             1|       50468.69|        0|     11|
|             1|       28183.78|        0|     12|
|             1|       75580.42|        0|     13|
|             1|        61836.9|        0|     14|
|             1|       25050.63|        0|     15|
|             1|       90974.63

### Placeholder

In [9]:
source = transactions_fg.select(["source","is_sar"]).read().withColumnRenamed("source","id")
target = transactions_fg.select(["target","is_sar"]).read().withColumnRenamed("target","id")
nodes = source.union(target)
sar_nodes = nodes.where(nodes.is_sar == 1)
non_sar = nodes.where(nodes.is_sar == 0)
non_sar_nodes = non_sar.join(sar_nodes, ["id"], "leftanti")
labels = sar_nodes.union(non_sar_nodes).dropDuplicates(subset=["id"])
labels.show()

+----+------+
|  id|is_sar|
+----+------+
|3997|     1|
|5300|     1|
| 463|     1|
|8086|     1|
|7833|     1|
|1088|     0|
|6654|     0|
|9465|     0|
|7880|     0|
|6336|     0|
|1580|     0|
|1829|     0|
|7993|     0|
|7240|     0|
|7253|     0|
|8389|     0|
|4935|     0|
|6658|     0|
|6466|     0|
|8592|     0|
+----+------+
only showing top 20 rows

In [10]:
accounts_df = accounts_df.withColumnRenamed("acct_id","id")
accounts_df.show()

+--------------+---------------+---------+---+
|tx_behavior_id|initial_deposit|prior_sar| id|
+--------------+---------------+---------+---+
|             1|       84442.19|        0|  0|
|             1|       75795.44|        0|  1|
|             1|       42057.16|        0|  2|
|             1|       25891.68|        0|  3|
|             1|       51127.47|        0|  4|
|             1|       40493.41|        0|  5|
|             1|       78379.86|        0|  6|
|             1|       30331.27|        0|  7|
|             1|        47659.7|        0|  8|
|             1|        58338.2|        0|  9|
|             1|       90811.29|        0| 10|
|             1|       50468.69|        0| 11|
|             1|       28183.78|        0| 12|
|             1|       75580.42|        0| 13|
|             1|        61836.9|        0| 14|
|             1|       25050.63|        0| 15|
|             1|       90974.63|        0| 16|
|             1|       98278.55|        0| 17|
|            

In [11]:
accounts_df.count()

10000

In [12]:
nodes_df = accounts_df.join(labels, ["id"])
nodes_df.count()

9998

In [13]:
node_td = fs.create_training_dataset(name="node_td",
                                       version=1,
                                       data_format="csv",   
                                       description="node training dataset")
node_td.save(nodes_df)